# Test running python in the background

Let's start by creating the resources we are going to need: a database and a container.

In [20]:
# jupyter nbconvert --to script InsertLoop.ipynb

import os
import time
import azure.cosmos
import azure.cosmos.cosmos_client as cosmos_client
from runpy import run_path

credentials = run_path("/home/cosmosuser/notebooks/credentials.py")

db_name = "mybgtest"
container_name = "myidcltest"

HOST = credentials['HOST']
MASTER_KEY = credentials['MASTER_KEY']
# with cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} ) as cosmos_client_id:
cosmos_client_id = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
# Create the database if it doesn't exist
try:
    database = cosmos_client_id.create_database_if_not_exists(db_name)
    print('Database "' + db_name + '" created')
    time.sleep(2)
except azure.cosmos.errors.CosmosHttpResponseError as e:
    if e.status_code == 409:
       print('Database already exists')
    else: 
        raise

# Reset the container
database_client = cosmos_client_id.get_database_client(db_name)
containers = database_client.list_containers()
if (any(container['id'] == container_name for container in containers)):
    database_client.delete_container(container_name)
    print('Container "' + container_name + '" dropped')
database_client.create_container(container_name, azure.cosmos.PartitionKey(path="/id"))
print('Container "' + container_name + '" created')

Database "mybgtest" created
Container "myidcltest" created


### Importing documents

We will bi scraping some data

In [25]:
import urllib.request  as urllib2
import json
import uuid


container_client = database_client.get_container_client(container_name)

for i in range(5):
    req = urllib2.Request("http://api.open-notify.org/iss-now.json")
    response = urllib2.urlopen(req)

    obj = json.loads(response.read())
    # {"iss_position": {"longitude": "-151.4464", "latitude": "-51.0588"}, "timestamp": 1579293714, "message": "success"}
    # print(obj)
 
    if obj['message'] == 'success':
        new_item = { 'id': str(uuid.uuid4()), 'timestamp': obj['timestamp'], 'latitude': obj['iss_position']['latitude'], 'longitude': obj['iss_position']['longitude'] }
        container_client.create_item(new_item)

        # print (obj['timestamp'])
        # print (obj['iss_position']['latitude'], obj['iss_position']['longitude'])
        
        time.sleep(5)
    else:
        print ('API failure: ', json.dumps(obj))
    print('.', end = '')



.....

# Declarations

In [22]:
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

### Testing a first query

Now that we have our test data, time to issue some queries! We start by fetching all documents with a `field1` value of `Mario86`:

In [24]:
query = 'SELECT VALUE COUNT(1) FROM c' # 'SELECT * FROM c'
start = time.time()
results = list(container_client.query_items(
    query=query,
    enable_cross_partition_query=True))
end = time.time()

print('Got ' + str(len(results)) + ' result(s): ' + json.dumps(results))
print('Time elapsed: %d ms'% ((end - start) * 1000))
print('Request charge: ' + container_client.client_connection.last_response_headers['x-ms-request-charge'] + ' RUs')

Got 1 result(s): [5]
Time elapsed: 32 ms
Request charge: 3 RUs


In [ ]:
if isnotebook():
    cosmos_client_id.delete_database(db_name)